In [1]:
%cd ..
from scripts.utils_latest import *

import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.colors as mcolors
import seaborn as sns
pd.set_option('display.max_columns', None)

/run/media/nazif/2F946E411BA61D49/thesis


In [2]:
df = pd.read_csv("results/1_generate_data.csv")
clash_df = clash()
df.head()

,mrna_start,mrna_end,pred_energy,mirna_start,mirna_end,mirna_dot_bracket_5to3,id,mirna_sequence,mirna_accession,enst,full_sequence_of_transcript,extended_mrna_sequence,extended_mrna_start,extended_mrna_end,clash_mrna_sequence,clash_mrna_start,clash_mrna_end,true_seed_type,true_num_basepairs,true_seed_basepairs,true_energy,alignment_string,pred_num_basepairs,pred_seed_basepairs,bp_difference,seed_bp_difference,mirna_length,mre_end,mre_start,mre_region
0,22,45,-27.7,0,20,)))))))))))))))))))).,0727A-1038930_1,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000340828,AGGGCAGGCGCGGCCCCTTCGGCTCCGAGCTGACCCTGATCAGGGC...,TTTAATATTTTTTTCTAGAAAACAGGTGACATTTGTATCTACGATA...,1760,1920,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,1790,1890,noncanonical_seed,20,6,-25.1,1111111111111111111100,20,6,0,0,22,1835,1813,TTATACAGAACCTACTGCCTCA
1,11,30,-26.2,0,18,))))))))).)))))))).,L1HS-1112536_1,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000343455,CGGAGGCGCGGCGCAGGCTGCTGCAGGCCCAGGTGAATGGAGTAAC...,TGCCAAGGAAATCAGCTAAATTACTACAAGCAGGAAATACCCGTGC...,3826,3958,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,3856,3928,9-mer,17,6,-24.4,1111111110111111110000,17,6,0,0,22,3886,3864,ACCCGTGCAACCAACTACCTCA
2,3,24,-23.6,0,21,.)))))).))))))))))))).,L2HS-818542_2,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000436639,GATTGCCAGGGCCGCCCTGTGCCCTCTGGCTCGGCGGTGGTGGGCG...,GCAGAACTCCTTTATGCTCTGAGAGCCATTACCCGCTATATGACCT...,2354,2464,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,2384,2434,noncanonical_seed,19,6,-22.2,0111111011111111111110,19,6,0,0,22,2408,2386,CCGCTATATGACCTGATGCCTT
3,3,24,-23.8,0,21,)))))))...)))))))))))),L2HS-1161339_2,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000282516,TCCGGTCGGCATTTTGTTCTGAGAGGGAGAGACGGAACGAGAGAGA...,AAATAAGACCCCAGCTCATGGTTAAACATGCAATGACTATGCAACC...,6539,6653,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,6569,6623,noncanonical_seed,18,6,-22.1,1111111000111111111111,19,6,1,0,22,6593,6571,ATGACTATGCAACCATACCTTA
4,2,24,-24.0,0,21,))))))))))))))...)))).,L2-407944_2,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000340139,AGGGACCACCGGGAACAGACGGATCGGCAGGGCGGGGCGGAACGGT...,ACAGCAAAATTGAGTTCAACAACACAAAACAATACTGGGAAACCTG...,1133,1238,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,1163,1208,noncanonical_seed,18,6,-21.9,1111111111111100011110,18,6,0,0,22,1187,1165,TACTGGGAAACCTGCTACTTCG


In [ ]:
negative_df = df[["mre_region", "mirna_sequence", "mirna_accession"]]

# shuffle mirna sequences using levenstein distance
for i, row in negative_df.iterrows():
    original_string = row['mirna_sequence']
    # exclude the original string from the column
    column = negative_df['mirna_sequence'].drop(i)
    new_string = find_most_different_string(original_string, column)
    negative_df.at[i, 'shuffled_sequence'] = new_string

In [ ]:
# renaming cols for convention
rename_dict = {"mirna_sequence": "old_mirna_sequence",
               "mirna_accession": "old_mirna_accession", "shuffled_sequence": "mirna_sequence"}
negative_df = negative_df.rename(columns=rename_dict)

In [ ]:
# adding accessions for new mirna sequences
accession_dict = clash_df.set_index('mirna_sequence')[
    'mirna_accession'].to_dict()
negative_df['mirna_accession'] = negative_df['mirna_sequence'].map(
    accession_dict)

In [ ]:
# renaming col to be able to run find_matches_with_rnaduplex()
negative_df = negative_df.rename(columns={"mre_region": "mrna_sequence"})

# finding matches on negative data
negative_df_results = find_matches_with_rnaduplex(negative_df)

# save checkpoint
negative_df.to_csv("results/3_negative_df_checkpoint.csv", index=False)
negative_df_results.to_csv("results/3_negative_df_results_checkpoint.csv", index=False)


In [3]:
# checkpoint 
negative_df = pd.read_csv("results/3_negative_df_checkpoint.csv")
negative_df_results = pd.read_csv("results/3_negative_df_results_checkpoint.csv")

# adding relevant columns
negative_df_results["mirna_sequence"] = negative_df["mirna_sequence"]
negative_df_results["mre_region"] = negative_df["mrna_sequence"]
negative_df_results["mirna_accession"] = negative_df["mirna_accession"]

# adding columns from CLASH
negative_df_results["id"] = clash_df["id"]
negative_df_results["enst"] = clash_df["enst"]
negative_df_results["extended_mrna_start"] = clash_df["extended_start"]
negative_df_results["extended_mrna_end"] = clash_df["extended_end"]
negative_df_results["extended_mrna_sequence"] = clash_df["extended_sequence"]
negative_df_results["clash_mrna_start"] = clash_df["true_start_index"]
negative_df_results["clash_mrna_end"] = clash_df["true_end_index"]
negative_df_results["full_sequence_of_transcript"] = clash_df["full_sequence_of_transcript"]


negative_df_results["mre_start"] = df["mre_start"]
negative_df_results["mre_end"] = df["mre_end"]




In [4]:
negative_df_results.head()

,mrna_start,mrna_end,pred_energy,mirna_start,mirna_end,mirna_dot_bracket_5to3,mirna_sequence,mre_region,mirna_accession,id,enst,extended_mrna_start,extended_mrna_end,extended_mrna_sequence,clash_mrna_start,clash_mrna_end,full_sequence_of_transcript,mre_start,mre_end
0,15,19,-2.4,8,12,.))).,CACGCTCATGCACACACCCACA,TTTATACAGAACCTACTGCCTC,MIMAT0003239,0727A-1038930_1,ENST00000340828,1760,1920,TTTAATATTTTTTTCTAGAAAACAGGTGACATTTGTATCTACGATA...,1790,1890,AGGGCAGGCGCGGCCCCTTCGGCTCCGAGCTGACCCTGATCAGGGC...,1813,1835
1,0,6,-7.5,7,13,.))))).,CACGCTCATGCACACACCCACA,CGTGCAACCAACTACCTCATAT,MIMAT0003239,L1HS-1112536_1,ENST00000343455,3826,3958,TGCCAAGGAAATCAGCTAAATTACTACAAGCAGGAAATACCCGTGC...,3856,3928,CGGAGGCGCGGCGCAGGCTGCTGCAGGCCCAGGTGAATGGAGTAAC...,3864,3886
2,6,18,-3.8,2,12,.))))).))).,CACGCTCATGCACACACCCACA,CGCTATATGACCTGATGCCTTT,MIMAT0003239,L2HS-818542_2,ENST00000436639,2354,2464,GCAGAACTCCTTTATGCTCTGAGAGCCATTACCCGCTATATGACCT...,2384,2434,GATTGCCAGGGCCGCCCTGTGCCCTCTGGCTCGGCGGTGGTGGGCG...,2386,2408
3,5,10,-3.9,7,12,.)))).,CACGCTCATGCACACACCCACA,TGACTATGCAACCATACCTTAC,MIMAT0003239,L2HS-1161339_2,ENST00000282516,6539,6653,AAATAAGACCCCAGCTCATGGTTAAACATGCAATGACTATGCAACC...,6569,6623,TCCGGTCGGCATTTTGTTCTGAGAGGGAGAGACGGAACGAGAGAGA...,6571,6593
4,2,15,-7.8,8,20,.)))....)))).,CACGCTCATGCACACACCCACA,TACTGGGAAACCTGCTACTTCG,MIMAT0003239,L2-407944_2,ENST00000340139,1133,1238,ACAGCAAAATTGAGTTCAACAACACAAAACAATACTGGGAAACCTG...,1163,1208,AGGGACCACCGGGAACAGACGGATCGGCAGGGCGGGGCGGAACGGT...,1165,1187


In [5]:
# export to csv
negative_df_results.to_csv("results/3_negative_data.csv", index=False)